# C/S架构

# 登录
新建一个对话框，包含两个文本框。
其中一个是密码框，只显示星号。
另外设置登录按钮。

## 新建数据库
新建名为medical_monitor的数据库。

字符编码选择utf8，增强对中文的支持。

![创建数据库](image/mysql_create_db.png)

## 新建用户
数据库新建用户，每位医生都需要有一个独立的用户名。
这里以doctor1为例。

新建Mysql用户的方法：
- 通过Mysql的管理程序实现
- 通过命令行实现
- 后续通过我们的Qt界面实现，在我们的程序里面完成用户的管理工作。

![新建用户](image/mysql_add_user.png)

## 连接
Qt程序尝试连接数据库，看是否能成功。

```cpp
#include <QtSql/QSqlDatabase>
// 加载驱动
QSqlDatabase db = QSqlDatabase::addDatabase("QMYSQL");
// 服务器的地址，可以是本地，也可以是远程的IP或者域名
db.setHostName("localhost");
// 数据库的名字
db.setDatabaseName("medical_monitor");
// 用户名和密码
db.setUserName("doctor1");
db.setPassword("123456");
// open方法打开与数据库的连接
// 如果连接成功返回true
// 服务器异常无法连接或者密码不对则返回false
bool ok = db.open();
```

运行时报错
```
QSqlDatabase: QMYSQL driver not loaded
QSqlDatabase: available drivers: QSQLITE QMYSQL QMYSQL3 QODBC QODBC3 QPSQL QPSQL7
```

搜索 StackOverflow 发现答案：
```
This error generally means that Qt MySQL plugin is fine (as it is listed in available drivers), but you are a missing the MySQL dll (thus preventing the driver to load).

The way to fix it is to place libmysql.dll somewhere in your PATH, e.g. adding the MySQL installation folder to PATH, or copy libmysql.dll in the same folder you have your exe in.
```

查看路径```C:\Qt\Qt5.9.9\5.9.9\mingw53_32\plugins\sqldrivers```，
确实存在```qsqlmysql.dll```文件，这是Qt框架连接数据库的插件。
说明缺少访问数据库的驱动。
我们安装的Qt可能是32位版本，而安装Mysql是64位。
需要下载32位驱动。

https://downloads.mysql.com/archives/c-c/

```libmysql.dll```文件放在：```C:\Qt\Qt5.9.9\5.9.9\mingw53_32\bin```。
需要与IDE里面的配置相同。
我们开发的程序会到这个路径查找加载动态链接库。

## 登录
以上用户名和密码部分都是固定的，用于开发测试。
但是真实环境不能写死，需要用界面的文本框获取，以验证用户是否有权限。

此部分已经在GUI章节学过，可以根据时间与掌握情况跳过。

# 取用户信息
取出用户的姓名，显示在界面上，用于提醒。

## 医生信息表
```python
class Doctor:
  mobile = '13812345678'
  name = '张三'
```

所以手机号是什么类型？字符串类型。
字符数呢？11位。

那么用整数型可以吗？
也可以，而且节省保存的字节数，查询也更快。

unsigned int的范围是0~4294967295，
不够11位，所以我们需要bigint，保存8字节，正好64位。
比用字符串保存还是要节省。


```sql
CREATE TABLE `medical_monitor`.`doctor` (
  `uid` VARCHAR(16) NOT NULL,
  `name` VARCHAR(4) NULL,
  `mobile` BIGINT UNSIGNED NULL,
  PRIMARY KEY (`uid`));
```

```sql
INSERT INTO medical_monitor.doctor
(uid, name, mobile)
VALUES
('doctor1', '张三', 13987654321)
```

一定要注意语句的引号，代表字符串的意思，与一般的编程语言一致。

之后再用查询语句验证效果。
```sql
SELECT * FROM doctor
```

取医生的姓名，需要执行一个SQL语句。

```cpp
#include <QtSql/QSqlQuery>
// 负责执行SQL语句的对象，
// 需要参数指明使用哪个连接，db是我们前面创建的数据库连接
QSqlQuery query(db);

// 执行的SQL
// 从doctor数据库查询，条件是用户名为doctor1
query.exec("SELECT name FROM doctor WHERE uid = 'doctor1'");

// 查询的结果类似excel一样的表格
// size返回查询结果的行数
qDebug() << "size:" << query.size(); 
```

有两种遍历结果集的方法：
1. 根据size进行for循环
2. 从第一个不断调用next方法后移进行遍历

第二种方法最为常用。

```cpp
// next 查询是否有下一行。
if(query.next())
{
    qDebug() << query.value("name");
}
else // 如果一开始就没有下一行，说明结果集为空
{
    qDebug() << "No such doctor";
}
```

打印的结果是 ``` QVariant(QString, "张三") ```
这里用到了QVariant变体类型，
因为数据库返回的类型不确定，可能是字符串、整数、日期……
变体意味着可以保存任何类型，同时也会指明实际保存的类型。
这种方法与python等脚本语言的变量类似。

```cpp
QString name = query.value("name").toString();
qDebug() << name;

int mobile = query.value("mobile").toInt();
```

接下来，可以把医生的姓名显示在界面上。

## SQL条件查询
之前执行SQL的时候，是写死的查询参数。
```cpp
query.exec("SELECT name FROM doctor WHERE uid = 'doctor1'");
```
实际应用需要根据登录的用户名修改。
修改方法比较简单：自行拼装SQL字符串。

```cpp
QString userid = 'doctor1';
QString sql = "SELECT name FROM doctor WHERE uid = '";
sql += userid;
sql += "'";
qDebug() << sql;
query.exec(sql);
```
字符串里面的单引号不能去掉，否则SQL就是非法的。

可见这种写法的可读性很差，把一个SQL语句分成好几段。
所以Qt提供了一种比较友好的方法：
```cpp
query.prepare("SELECT name FROM doctor WHERE uid = :id");
query.bindValue(":id", "doctor1");
//或者
query.bindValue(":id", userid);

//最后
query.exec();
```

这种方法是从prepare开始，新建一个SQL语句。
需要参数的地方用冒号加变量的方式表示。
之后再用bindValue方法把变量替换成实际的值。
这里有一个细节，原始的SQL语句的变量两侧并没有加引号，
Qt组装SQL语句的时候会自动处理。